TLDR: 
1. Purpose: A demo of **tool use** to do a simple math problem to calculate the cosine of 8
2. Model name: `anthropic--claude-3-opus`
3. Model Provider: Amazon Bedrock
4. Only one tool function is used
5. Reference: [Amazon Bedrock](https://community.aws/content/2hW5367isgQOkkXLYjp4JB3Pe16/intro-to-tool-use-with-the-amazon-bedrock-converse-api?lang=en)

In [1]:
import math
from pprint import pprint

from gen_ai_hub.proxy.native.amazon.clients import Session

model_name = 'anthropic--claude-3-opus'
bedrock = Session().client(model_name=model_name)

In [2]:
tool_list = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x",
            "inputSchema": {
                "json": {
                    "type": "object", 
                    "properties": {
                        "x": {
                            "type": "number", 
                            "description": "The number to pass to the function"
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    }, 
    {
        "toolSpec": {
            "name": "get_numerical",
            "description":  "Extracts only the numerical value from a given answer, ignoring explanations or other text.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                    "answer": {
                            "type": "string",
                            "description": "The input string that may contain a number along with other text. This function extracts the numerical value."
                        }
                    },
                    "required": ["answer"]
                }
            }
        }
    },
]

In [7]:
def call_bedrock(message_list, tool_list):
    bedrock = Session().client(model_name=model_name)
    response = bedrock.converse(
        messages = message_list, 
        inferenceConfig = {
            "maxTokens": 4096, 
            "temperature": 0.0, 
        },
        toolConfig={
            "tools": tool_list
        }
    )
    return response

In [8]:
message_list = []
initial_message = {
    "role": "user", 
    "content": [
        {"text": "What is the cosine of 8? Give me only the numerical number"}
    ]
}
message_list.append(initial_message)

In [9]:
print(message_list)

[{'role': 'user', 'content': [{'text': 'What is the cosine of 8? Give me only the numerical number'}]}]


In [10]:
# Call 1
response = call_bedrock(message_list, tool_list)

In [11]:
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '672',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 02 Dec 2024 17:22:18 GMT'},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 7288},
 'output': {'message': {'content': [{'text': '<thinking>\n'
                                             'The cosine tool is relevant for '
                                             'calculating the cosine of a '
                                             'number. It requires one '
                                             'parameter:\n'
                                             '- x (number): The number to pass '
                                             'to the function\n'
                                             'The user has directly provided '
                                             'the value for x, which is 8. No '
         

In [12]:
response_message = response['output']['message']
pprint(response_message)

{'content': [{'text': '<thinking>\n'
                      'The cosine tool is relevant for calculating the cosine '
                      'of a number. It requires one parameter:\n'
                      '- x (number): The number to pass to the function\n'
                      'The user has directly provided the value for x, which '
                      'is 8. No other tools are needed.\n'
                      'The user also specified to return only the numerical '
                      'value, so the get_numerical tool should be called on '
                      'the result of the cosine tool.\n'
                      '</thinking>'},
             {'toolUse': {'input': {'x': 8},
                          'name': 'cosine',
                          'toolUseId': 'tooluse_i24KRwHJTNiWGS8Y6xK9zw'}}],
 'role': 'assistant'}


In [13]:
response_content_blocks = response_message['content']
pprint(response_content_blocks)

[{'text': '<thinking>\n'
          'The cosine tool is relevant for calculating the cosine of a number. '
          'It requires one parameter:\n'
          '- x (number): The number to pass to the function\n'
          'The user has directly provided the value for x, which is 8. No '
          'other tools are needed.\n'
          'The user also specified to return only the numerical value, so the '
          'get_numerical tool should be called on the result of the cosine '
          'tool.\n'
          '</thinking>'},
 {'toolUse': {'input': {'x': 8},
              'name': 'cosine',
              'toolUseId': 'tooluse_i24KRwHJTNiWGS8Y6xK9zw'}}]


In [14]:
message_list.append(response_message)
pprint(message_list)

[{'content': [{'text': 'What is the cosine of 8? Give me only the numerical '
                       'number'}],
  'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'The cosine tool is relevant for calculating the cosine '
                       'of a number. It requires one parameter:\n'
                       '- x (number): The number to pass to the function\n'
                       'The user has directly provided the value for x, which '
                       'is 8. No other tools are needed.\n'
                       'The user also specified to return only the numerical '
                       'value, so the get_numerical tool should be called on '
                       'the result of the cosine tool.\n'
                       '</thinking>'},
              {'toolUse': {'input': {'x': 8},
                           'name': 'cosine',
                           'toolUseId': 'tooluse_i24KRwHJTNiWGS8Y6xK9zw'}}],
  'role': 'assistant'}]


In [15]:
follow_up_content_blocks = []

for content_block in response_content_blocks:
    if 'toolUse' in content_block:
        tool_use_block = content_block['toolUse']
        tool_use_id = tool_use_block['toolUseId']
        tool_use_name = tool_use_block['name']

        if tool_use_name == 'cosine':
            x = tool_use_block['input']['x']
            tool_result_value = math.cos(x)

            follow_up_content_blocks.append({
                    "toolResult": {
                        'toolUseId': tool_use_id,
                        'content':[
                            {
                                "json": {
                                    "result": tool_result_value
                                }
                            }
                        ]
                    }
                })

pprint(follow_up_content_blocks)

if len(follow_up_content_blocks) > 0:

    follow_up_message = {
        "role": "user", 
        "content": follow_up_content_blocks,
    }

    message_list.append(follow_up_message)

pprint(message_list)

[{'toolResult': {'content': [{'json': {'result': -0.14550003380861354}}],
                 'toolUseId': 'tooluse_i24KRwHJTNiWGS8Y6xK9zw'}}]
[{'content': [{'text': 'What is the cosine of 8? Give me only the numerical '
                       'number'}],
  'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'The cosine tool is relevant for calculating the cosine '
                       'of a number. It requires one parameter:\n'
                       '- x (number): The number to pass to the function\n'
                       'The user has directly provided the value for x, which '
                       'is 8. No other tools are needed.\n'
                       'The user also specified to return only the numerical '
                       'value, so the get_numerical tool should be called on '
                       'the result of the cosine tool.\n'
                       '</thinking>'},
              {'toolUse': {'input': {'x': 8},
                         

In [16]:
# Call 2
response = call_bedrock(message_list, tool_list)

In [17]:
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '297',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 02 Dec 2024 17:23:05 GMT'},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 2250},
 'output': {'message': {'content': [{'toolUse': {'input': {'answer': '-0.14550003380861354'},
                                                 'name': 'get_numerical',
                                                 'toolUseId': 'tooluse_xAAVxpn0Q3ScSXovJUqWTQ'}}],
                        'role': 'assistant'}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 878, 'outputTokens': 62, 'totalTokens': 940}}


In [18]:
response_message = response['output']['message']
response_content_blocks = response_message['content']
pprint(response_content_blocks)

response_message = response['output']['message']
message_list.append(response_message)
pprint(message_list)

[{'toolUse': {'input': {'answer': '-0.14550003380861354'},
              'name': 'get_numerical',
              'toolUseId': 'tooluse_xAAVxpn0Q3ScSXovJUqWTQ'}}]
[{'content': [{'text': 'What is the cosine of 8? Give me only the numerical '
                       'number'}],
  'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'The cosine tool is relevant for calculating the cosine '
                       'of a number. It requires one parameter:\n'
                       '- x (number): The number to pass to the function\n'
                       'The user has directly provided the value for x, which '
                       'is 8. No other tools are needed.\n'
                       'The user also specified to return only the numerical '
                       'value, so the get_numerical tool should be called on '
                       'the result of the cosine tool.\n'
                       '</thinking>'},
              {'toolUse': {'input': {'x': 8},
    

In [19]:
follow_up_content_blocks = []

for content_block in response_content_blocks:
    if 'toolUse' in content_block:
        tool_use_block = content_block['toolUse']
        tool_use_id = tool_use_block['toolUseId']
        tool_use_name = tool_use_block['name']

        if tool_use_name == 'get_numerical':
            answer = tool_use_block['input']['answer']
            try:
                answer = float(answer)
                tool_result_value = answer
            except:
                raise ValueError(f"Invalid input for numerical conversion: {answer}")
                
            follow_up_content_blocks.append({
                    "toolResult": {
                        'toolUseId': tool_use_id,
                        'content':[
                            {
                                "json": {
                                    "result": tool_result_value
                                }
                            }
                        ]
                    }
                })

if len(follow_up_content_blocks) > 0:
    follow_up_message = {
        "role": "user", 
        "content": follow_up_content_blocks,
    }

    message_list.append(follow_up_message)

pprint(message_list)

[{'content': [{'text': 'What is the cosine of 8? Give me only the numerical '
                       'number'}],
  'role': 'user'},
 {'content': [{'text': '<thinking>\n'
                       'The cosine tool is relevant for calculating the cosine '
                       'of a number. It requires one parameter:\n'
                       '- x (number): The number to pass to the function\n'
                       'The user has directly provided the value for x, which '
                       'is 8. No other tools are needed.\n'
                       'The user also specified to return only the numerical '
                       'value, so the get_numerical tool should be called on '
                       'the result of the cosine tool.\n'
                       '</thinking>'},
              {'toolUse': {'input': {'x': 8},
                           'name': 'cosine',
                           'toolUseId': 'tooluse_i24KRwHJTNiWGS8Y6xK9zw'}}],
  'role': 'assistant'},
 {'content': [{'too

In [20]:
response = call_bedrock(message_list, tool_list)

In [21]:
pprint(response)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '205',
                                      'content-type': 'application/json',
                                      'date': 'Mon, 02 Dec 2024 17:23:28 GMT'},
                      'HTTPStatusCode': 200,
                      'RetryAttempts': 0},
 'metrics': {'latencyMs': 2247},
 'output': {'message': {'content': [{'text': '-0.14550003380861354'}],
                        'role': 'assistant'}},
 'stopReason': 'end_turn',
 'usage': {'inputTokens': 963, 'outputTokens': 13, 'totalTokens': 976}}
